# Using CLIPSeg with Hugging Face Transformers

Using Hugging Face Transformers, you can easily download and run a pre-trained CLIPSeg model on your images. Let’s start by installing transformers.

In [1]:
# # Check nvcc version
# !nvcc -V
# # Check GCC version
# !gcc --version

# # install dependencies: (use cu111 because colab has CUDA 11.1)
# %pip install -U openmim
# !mim install "mmengine>=0.7.0"
# !mim install "mmcv>=2.0.0rc4"

# # Install mmdetection
# !rm -rf mmdetection
# !git clone https://github.com/open-mmlab/mmdetection.git
# %cd mmdetection

# %pip install -e .

# !pip install torch opencv-python Pillow
# !pip install -q 'git+https://github.com/facebookresearch/segment-anything.git'
# !pip install -q jupyter_bbox_widget roboflow dataclasses-json supervision
# !pip install -q transformers
# !wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

# !pip uninstall -y supervision
# !pip install -q supervision==0.6.0
!pip install -q jupyter_bbox_widget roboflow dataclasses-json supervision


In [1]:
# import sys
# sys.path.append('/content')

# from google.colab import drive
# drive.mount('/content/drive')

# import supervision as sv
# print(sv.__version__)
import torch

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



In [2]:
#GLIP TINY
import os

HOME = "/home/hongxin/"
model_name =  'glip_atss_swin-t_fpn_dyhead_pretrain_obj365-goldg-cc3m-sub'
GLIP_CHECKPOINT_PATH = os.path.join(HOME, "weights", "glip_tiny_mmdet-c24ce662.pth")
!mim download mmdet --config {model_name} --dest {GLIP_CHECKPOINT_PATH}
checkpoint = GLIP_CHECKPOINT_PATH
#GLIP LARGE
# model_name = 'glip_atss_swin-l_fpn_dyhead_pretrain_mixeddata'
# checkpoint =  '/content/checkpoints/glip_l_mmdet-abfe026b.pth'
# !mkdir /content/checkpoints
# !mim download mmdet --config {modelName} --dest /content/checkpoints



processing glip_atss_swin-t_fpn_dyhead_pretrain_obj365-goldg-cc3m-sub...
glip_tiny_mmdet-c24ce662.pth exists in /home/hongxin/weights/glip_tiny_mmdet-c24ce662.pth
Successfully dumped glip_atss_swin-t_fpn_dyhead_pretrain_obj365-goldg-cc3m-sub.py to /home/hongxin/weights/glip_tiny_mmdet-c24ce662.pth


In [5]:
from mmdet.apis import DetInferencer

# Set the device to be used for evaluation
device = 'cuda:0'
# import sys
# sys.path.append(os.path.join(HOME, "weights"))
# Initialize the DetInferencer
inferencer = DetInferencer(model_name, checkpoint, device)


# #TEST
# # Use the detector to do inference
# img = '/content/drive/MyDrive/CSCI 567 /segment-anything/datasets/people_poses/val_images/100034_483681.jpg'
# result = inferencer(img, texts='Socks', out_dir='./output')
# predictions = result['predictions']

# bbox = []
# if len(predictions) > 0:
#   bboxVal = predictions[0]['bboxes']
#   xMin = 0.0
#   yMin = 0.0
#   xMax = 0.0
#   yMax = 0.0
#   if len(bboxVal) > 0:
#     xMin, yMin, xMax, yMax = bboxVal[0]
#   i = 0
#   for box in bboxVal:
#     if predictions[0]['scores'][i] > 0.5:
#       print(box)
#       x1, y1, x2, y2 = box
#       if x1 < xMin:
#         xMin = x1
#       if y1 < yMin:
#         yMin = y1
#       if x2 > xMax:
#         xMax = x2
#       if y2 > yMax:
#         yMax = y2
#     i += 1

#   bbox = [xMin, yMin, xMax, yMax]


# print("final")
# print(bbox)

# # Show the output image
# from PIL import Image
# Image.open('./output/vis/100034_483681.jpg')


Loads checkpoint by local backend from path: /home/hongxin/weights/glip_tiny_mmdet-c24ce662.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: language_model.language_backbone.body.model.embeddings.position_ids



/home/hongxin/mmdetection/mmdet/apis/det_inferencer.py:130: UserWarning: dataset_meta or class names are not saved in the checkpoint's meta data, use COCO classes by default.
  warnings.warn(


12/05 02:24:32 - mmengine - WARNING - Failed to search registry with scope "mmdet" in the "function" registry tree. As a workaround, the current "function" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmdet" is a correct scope, or whether the registry is initialized.


/home/hongxin/anaconda3/envs/CSCI567/lib/python3.11/site-packages/mmengine/visualization/visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.LocalVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '


## Text prompting

Let’s start by defining some text categories we want to segment.

Now that we have our inputs, we can process them and input them to the model.

In [6]:
def getPredictions(promptValue, image, imagePath):
  result = inferencer(imagePath, texts= promptValue)
  predictions = result['predictions']

  bbox = []
  if len(predictions) > 0:
    bboxVal = predictions[0]['bboxes']
    print(predictions[0]['scores'])
    xMin = 0.0
    yMin = 0.0
    xMax = 0.0
    yMax = 0.0
    if len(bboxVal) > 0:
      xMin, yMin, xMax, yMax = bboxVal[0]
    i = 0
    for box in bboxVal:
      if predictions[0]['scores'][i] > 0.5:
        x1, y1, x2, y2 = box
        if x1 < xMin:
          xMin = x1
        if y1 < yMin:
          yMin = y1
        if x2 > xMax:
          xMax = x2
        if y2 > yMax:
          yMax = y2
      i += 1
    bbox = [xMin, yMin, xMax, yMax]


  if len(bbox) > 0:
    return bbox
  return [0, 0, 0, 0]

# END OF CHANGES
NO CODE BELOW THIS SECTION WAS CHANGED


In [7]:
from PIL import Image

def make_square_by_padding(img, fill_color=(0, 0, 0)):
    # img = Image.open(requests.get(url, stream=True).raw)
    width, height = img.size

    # Determine the size for the square
    new_size = max(width, height)

    # Create a new image with the desired size and fill color
    new_img = Image.new("RGB", (new_size, new_size), fill_color)

    # Paste the original image onto the center of the new image
    new_img.paste(img, ((new_size - width) // 2, (new_size - height) // 2))

    return new_img



In [8]:
import cv2
import matplotlib.pyplot as plt
from torchvision.ops import masks_to_boxes
import torchvision.transforms as T
import torchvision.transforms.functional as F

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image, Image, ImageDraw
import numpy as np

def processPredictionImage(pred, img):
    processed_tensor = torch.sigmoid(torch.reshape(torch.tensor(pred), (352, 352)))
    image_np = (processed_tensor.detach().numpy() * 255).astype(np.uint8)

    _, binary_image = cv2.threshold(image_np, 127, 255, cv2.THRESH_BINARY)

    desired_width, desired_height = img.size
    resized_image = cv2.resize(binary_image, (max(desired_width, desired_height),  max(desired_width, desired_height)), interpolation=cv2.INTER_AREA)

    # Get dimensions of the binary image
    height, width = resized_image.shape[:2]

    # Check if the desired crop size is smaller than the original image size
    if desired_width <= width and desired_height <= height:
        # Calculate the top-left corner of the crop
        x = width // 2 - desired_width // 2
        y = height // 2 - desired_height // 2

        # Crop the image
        cropped_image = resized_image[y:y+desired_height, x:x+desired_width]
        return cropped_image

    else:
        print("failed to crop image")
        return resized_image

In [9]:

def getBoundingBox(predictionImage):
    contours, _ = cv2.findContours(predictionImage, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        list_of_pts = []
        for ctr in contours:
            list_of_pts += [pt[0] for pt in ctr]
        ctr = np.array(list_of_pts).reshape((-1,1,2)).astype(np.int32)
        # largest_contour = max(cv2.convexHull(ctr), key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(cv2.convexHull(ctr))

        print(f"Bounding Box: x={x}, y={y}, width={(w)}, height={(h)}")
        return [x, y, (w + x), (h + y)]

    else:
        print("No contours found")
        return [0, 0, 0, 0]



# SAM Integration

In [ ]:
!pip install torch opencv-python Pillow
!pip install git+https://github.com/openai/CLIP.git
!pip install -q 'git+https://github.com/facebookresearch/segment-anything.git'
!pip install -q jupyter_bbox_widget roboflow dataclasses-json supervision

!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

In [12]:
from segment_anything import sam_model_registry, SamPredictor

sam_checkpoint = "../../sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

predictor = SamPredictor(sam)

In [13]:
def getSAMPreditction(image, box):

    image_np = np.array(image)
    image2 = cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB)
    predictor.set_image(image2)

    input_box = np.array(box)
    if(input_box[0] == 0 and input_box[1] == 0 and input_box[2] == 0 and input_box[3] == 0):
        masks, _, _ = predictor.predict(
            point_coords=None,
            point_labels=None,
            box=None,
            multimask_output=False,
        )
    else:
        masks, _, _ = predictor.predict(
            point_coords=None,
            point_labels=None,
            box=input_box[None, :],
            multimask_output=False,
        )
    return masks[0]

In [14]:

# numpy version
def pixAcc(predicted, target):
    same = (predicted == target).sum()
    w, h = target.shape
    print("Target seg shape: {}, Predicted seg shape: {}, #Same pixels: {}".format(target.shape, predicted.shape, same))
    return same / (w * h)

# input: bool matrix
def IOU(predicted , target):
    intersection = np.logical_and(target, predicted).sum()
    union = np.logical_or(target, predicted).sum()
    if union == 0:
        iou_score = 0
    else :
        iou_score = intersection / union
    return iou_score

In [15]:

def gt_to_anns_of_label_mask(mask_gt):
  labels = np.unique(mask_gt)
  anns = []
  for label in labels:
    # skip background
      if label == 0:
          continue
      mask = np.all(mask_gt == label, axis=-1)
      anns.append({
        'area': np.sum(mask),
        'segmentation': mask,
        'label': label,
      })
  return anns

# Final Results

In [16]:
import numpy as np

def getFinalPredictions(images, imagePaths, promptVals):
  preds = []
  boundingBoxes = []
  masks = []
  emptyPrompt = ""
  i = 0
  for img in images:
    imgResults = {}
    imageBoundingBoxes = {}
    imageMasks = {}
    img_with_border = make_square_by_padding(img)

    for prompt in promptVals:
      # prediction = getPredictions(prompt, img)
  #     imgResults[prompt] = prediction
  #     predictionImage = processPredictionImage(prediction, img)
      print(prompt)
      bbox = getPredictions(prompt, img, imagePaths[i])

      imageBoundingBoxes[prompt] = bbox
      print(bbox)
      if(bbox[0] == 0 and bbox[1] == 0 and bbox[2] == 0 and bbox[3] == 0 and emptyPrompt == ""):
        print("Here")
        mask = getSAMPreditction(img, bbox)
        imageMasks[prompt] = mask
        emptyPrompt = prompt
      elif(bbox[0] == 0 and bbox[1] == 0 and bbox[2] == 0 and bbox[3] == 0):
        print("Here")
        imageMasks[prompt] = imageMasks[emptyPrompt]
      else:
        mask = getSAMPreditction(img, bbox)
        imageMasks[prompt] = mask

    emptyPrompt = ""
    preds.append(imgResults)
    boundingBoxes.append(imageBoundingBoxes)
    masks.append(imageMasks)
    i += 1
  preds = np.array(preds)
  boundingBoxes = np.array(boundingBoxes)
  return boundingBoxes, preds, masks


In [ ]:
import os
from tqdm import tqdm
import copy

def getImages(num):
    images = []
    imagePaths = []
    truthMasks = []
    data_list = []
    root = ""
    root = "/content/drive/MyDrive/CSCI 567 /segment-anything/datasets/people_poses/"
    textFile = "val_id.short.txt"
    imageFile = "val_images/"
    segmentationFile = "val_segmentations/"
    with open(os.path.join(root, f"val_id.short.txt"), 'r') as lf:
        data_list = [ s.strip() for s in lf.readlines() ]

    num_valid_case, sum_miou, sum_pixAcc = 0,0,0
    try:
        for data_name in (pbar := tqdm(data_list[:num])):
            img_path = root + imageFile + data_name + '.jpg'
            seg_path = root +  segmentationFile+ data_name + '.png'

            # Read Image and Ground truth mask
            img = copy.deepcopy(Image.open(img_path))
            imagePaths.append(img_path)
            # display(img)
            if img is None:
                print("\nimage is None", data_name)
                continue
            else:
                images.append(img)
            mask_gt = cv2.imread(seg_path)
            if mask_gt is None:
                print("\nmask_gt is None", data_name)
                continue
            else:
                truthMasks.append(mask_gt)
    except Exception as e:
        print("ERROR")
        print(e)
    return images, truthMasks, imagePaths


In [ ]:
def get_masked_image(original_image, segmentation):
  # Visualize

  overlay_image = Image.new('RGBA', original_image.size, (0, 0, 0, 0))
  overlay_color = (255, 0, 0, 200)

  draw = ImageDraw.Draw(overlay_image)
  segmentation_mask_image = Image.fromarray(segmentation.astype('uint8') * 255)
  draw.bitmap((0, 0), segmentation_mask_image, fill=overlay_color)

  return Image.alpha_composite(original_image.convert('RGBA'), overlay_image)

In [ ]:

def evaluate(images, masks, truthMasks):
  # Get confidence scores for each masks generated by SAM for
  # each object label existing in the given image

  results = []
  label_name = ["Background","Hat","Hair","Glove",
        "Sunglasses","UpperClothes","Dress","Coat","Socks","Pants",
        "Jumpsuits","Scarf","Skirt","Face","Left-arm","Right-arm","Left-leg","Right-leg","Left-shoe","Right-shoe"]


  for i in range(len(truthMasks)):
    anns = gt_to_anns_of_label_mask(truthMasks[i])
    for ann in anns:
      mask = masks[i]
      image = images[i]
      label =  label_name[ann['label']]
      print(label_name[ann['label']])
      # Get Corresponding gt Mask, generated Mask for Evaluation
      print(mask[label])
      iou = IOU(mask[label],ann['segmentation'])
      print(iou)
      result_image = get_masked_image(image, mask[label])
      # display(result_image)
      pixacc = pixAcc(mask[label],ann['segmentation'])
      result = {
          "iou": iou,
          "pixacc": pixacc,
          "label_num": ann["label"],
          "label_name": label,
          "masked_img": result_image,
          "pred_mask": mask[label],
          "gt_mask": ann
      }
      print("iou:{}, pixacc:{}, label num:{}, label_name:{}".format(iou, pixacc, ann["label"], label))
      results.append(result)

  return results

# MAIN TRAINING

In [ ]:
import numpy as np

images, truthMasks, imagePaths = getImages(32)

bodyPrompts = ["Background","Hat","Hair","Glove", "Sunglasses","UpperClothes","Dress","Coat","Socks","Pants", "Jumpsuits","Scarf","Skirt","Face","Left-arm","Right-arm","Left-leg","Right-leg","Left-shoe","Right-shoe"]
# bodyPrompts = ["Socks", "Hat"]

boundingBoxes, preds, masks = getFinalPredictions(images, imagePaths, bodyPrompts)



# SAVING DATA

In [ ]:
def compute_pix_acc(predicted, target):

    assert predicted.shape == target.shape
    assert len(predicted.shape) == 2
    return (predicted == target).mean()

def compute_IOU(predicted, target):

    assert predicted.shape == target.shape

    assert len(predicted.shape) == 2
    intersection = np.logical_and(target, predicted).sum()
    union = np.logical_or(target, predicted).sum()
    assert union > 0
    return intersection / union
def compute_metric(name, masks, label):
    """ name: data_id
        mask: { label_id: numpy.ndarray(shape=(H, W)) }
        label: np.ndarray(shape=(H, W)) --> numbers from 0 to 19
    """
    pix_acc_metric = { "name": name }
    iou_metric = { "name": name }
    empty = np.zeros_like(label)
    LABELS = ["Background","Hat","Hair","Glove",
        "Sunglasses","UpperClothes","Dress","Coat","Socks","Pants",
        "Jumpsuits","Scarf","Skirt","Face","Left-arm","Right-arm","Left-leg","Right-leg","Left-shoe","Right-shoe"]

    for i, label_name in enumerate(LABELS):
        mask_i = masks.get(label_name, empty)
        label_i = (label == i)
        if label_i.sum() == 0:
            # pandas dataframe automatically skips nan
            # when computing .count() and .mean()
            iou_metric[label_name] = np.nan
            pix_acc_metric[label_name] = np.nan
        else:
            iou_metric[label_name] = compute_IOU(mask_i, label_i)
            pix_acc_metric[label_name] = compute_pix_acc(mask_i, label_i)

    return iou_metric, pix_acc_metric

In [ ]:
def getImages2(num):
    images = []
    truthMasks = []
    data_list2 = []
    root = ""
    # root = "/content/drive/MyDrive/CSCI567/segment-anything/datasets/people_poses/"
    textFile = "val_id.short.txt"
    imageFile = "val_images/"
    segmentationFile = "val_segmentations/"
    with open(os.path.join(root, f"val_id.short.txt"), 'r') as lf:
        data_list = [ s.strip() for s in lf.readlines() ]

    num_valid_case, sum_miou, sum_pixAcc = 0,0,0
    try:
        for data_name in (pbar := tqdm(data_list[:num])):
            img_path = root + imageFile + data_name + '.jpg'
            seg_path = root +  segmentationFile+ data_name + '.png'

            # Read Image and Ground truth mask
            img = copy.deepcopy(Image.open(img_path))

            # display(img)
            if img is None:
                print("\nimage is None", data_name)
                continue
            else:
                data_list2.append(data_name)
                images.append(img)
            mask_gt = cv2.imread(seg_path)
            if mask_gt is None:
                print("\nmask_gt is None", data_name)
                continue
            else:
                truthMasks.append(mask_gt)
    except Exception as e:
        print("ERROR")
        print(e)
    return images, truthMasks, data_list2


In [ ]:
# import pandas as pd
# results = evaluate(images, masks, truthMasks)
from PIL import Image, Image, ImageDraw
from torch.jit import Error
import pandas as pd
# numpy metrics
import os
from tqdm import tqdm
# root = "/content/drive/MyDrive/CSCI567/segment-anything/datasets/people_poses/"
root = ""
prompt = "The object of "

# try:
miou_table = []
pix_acc_table = []
images, truthMasks, data_list = getImages2(32)
print(len(truthMasks))
print(len(images))
print(len(data_list))

result_table = {}
i = 0
try:
  for data_name in data_list:

    anns = gt_to_anns_of_label_mask(truthMasks[i])
    miou, pix_acc = compute_metric(data_name, masks[i], truthMasks[i][:,:,0])
    miou_table.append(miou)
    pix_acc_table.append(pix_acc)
    predict_anns = []
    for ann in anns:

      ## Get Label Index with Highest Score
      predict_anns.append({
          'segmentation': masks[i],
          'label': ann['label'],
          'gt': ann['segmentation'],
      })
    i += 1
    result_table[data_name] = predict_anns


except Exception as e:
  print(e)
  print(miou_table)
  print(pix_acc_table)

miou_table = pd.DataFrame(miou_table, columns=miou_table[0].keys()).set_index('name')
miou_table.to_csv('/Users/arianasokolov/Desktop/ClipSegSam/CSVs/miou.csv')
np.save('vis_cliseg_sam_32.npy', result_table)

pix_acc_table = pd.DataFrame(pix_acc_table, columns=pix_acc_table[0].keys()).set_index('name')
pix_acc_table.to_csv('/Users/arianasokolov/Desktop/ClipSegSam/CSVs/pix_acc.csv')

# VISUALIZATION

EVERYTHING BELOW THIS SECTION IS JUST TO TEST THINGS AND DOESN'T MATTER

In [ ]:
print(result_table)

In [ ]:
def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)
def get_mask(mask,color=None):
    if color is None:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    # else:
    #     color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    return mask_image

def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    return plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0,0,0,0), lw=2)

In [ ]:
prompts = bodyPrompts[:2]
last = 1
# *****
# Setup plot
_, ax = plt.subplots(len(boundingBoxes[:last]), len(prompts) + 1, figsize=(3*(len(prompts) + 1), 4))
[a.axis('off') for a in ax.flatten()]

for i in range(len(boundingBoxes[:last])):
  # Show padded image
  print(imagePaths[i])
  ax[0].imshow(images[i])

  for j in range(len(prompts)):
    # Show mask
    print(prompts[j])
    print(preds[i])
    # predictionImage = processPredictionImage(preds[i][prompts[j]], images[i])
    # ax[i][j+1].imshow(predictionImage)
    # ax[i][j+1].imshow(get_mask(masks[0]["Socks"]))

    # Show bounding box
    print(boundingBoxes[i][prompts[j]])
    ax[0].add_patch(show_box(boundingBoxes[i][prompts[j]], plt.gca()))


    # Show prompt
    # ax[i][j+1].text(0, -15, prompts[j])

In [ ]:
input_box = np.array(boundingBoxes[0]["Socks"])


plt.figure(figsize=(10, 10))
plt.imshow(images[0])
show_mask(masks[0]["Socks"], plt.gca())
# plt.gca().add_patch(show_box(input_box, plt.gca()))
plt.axis('off')
plt.show()